# [SOLO: Segmenting Objects by Locations](../papers/solo/1912.04488.pdf)

## Highlights

SOLO takes an image as input, directly outputs instance masks and corresponding class probabilities, in a fully convolutional, box-free and grouping-free paradigm.

SOLO proposes to segment instance according to their center location (one per grid cell as for YOLO).

**NOTE**: The approach can be generalized to other type of tasks (i.e. instance contour detection).

**WARNING**: This approach assumes that each cell of the grid must belong to one individual instance (thus instance category).

![](../assets/solo-1.png)

SOLO directly learns with the instance mask annotations solely during training, and predicts instance masks end-to-end without grouping post-processing.

### Problem Formulation

![](../assets/solo-2.png)

The central idea of SOLO framework is to reformulate the instance segmentation as two simultaneous category-aware prediction problems. Concretely, our system divides the input image into a uniform grids, i.e., $S \times S$. If the center of an object falls into a grid cell, that grid cell is responsible for:
1) predicting the semantic category as well as
2) segmenting that object instance.

#### Semantic Category

For each grid, SOLO predicts the C-dimensional output to indicate the semantic class probabilities, where C is the number of classes.

#### Instance Mask

Each positive grid cell will also generate the corresponding instance mask. The instance mask output will have $H \times W \times S^2$ dimension.

A one-to-one correspondance is established between the semantic category and class-agnostic mask.

#### Spatial variance

As segmentation masks are conditionned on the grid cells, the model needs to be *spatially variant*. Unlike *CoordConv*, the input image is concatenated with the coordinates of each pixel, normalized within $\left[-1, 1 \right]$. Thus, the input tensor becomes of dimension $H \times W \times (D+2)$, where the last two elements are the $x$-$y$ pixel coordinates.

![](../assets/solo-3.png)

### Network Architecture

SOLO is made of:
- a convolutional backbone
- a FPN (with 256-d channels)
- preadiction head: semantic category and instance mask (see figure). **Weights are shared across different levels**. Only the last convolution is not shared (???).

### Learning

#### Label Assignment

A ground truth mask is assigned to a cell if its center (plus a margin) falls in it. Valid cells are cells whose center lays in $\left( c_x, c_y, \epsilon w, \epsilon h \right)$ with $\epsilon = 0.2$. For each positive samples, the corresponding target binary mask is annotated.

#### Loss Function

$L = L_{cate} + \lambda L_{mask}$ where $L_{cate}$ is the conventional focal loss for semantic category classification and $L_{mask}$ is the loss for mask prediction (see paper) using the Dice loss (after comparison with BCE and focal loss).

#### Inference

1) Forward through the network
2) Filter out predictions with confidence lower than 0.1
3) Compute confidence score: compute maskness (average of pixel with softmax greather than 0.5) and multiply by classification score
4) Select top 500 scoring masks and feed NMS operation, with a threshold of 0.5 to convert from predicted soft masks to binary masks

### Experiments

See paper for training details

## Code

Training code is available here: [https://github.com/aim-uofa/AdelaiDet](https://github.com/aim-uofa/AdelaiDet).